In [32]:
#Zeri di funzione
import numpy as np
import SolveTriangular as st
import scipy as sp

import math
def sign(x):
  """
  Funzione segno che restituisce 1 se x è positivo, 0 se x è zero e -1 se x è negativo.
  """
  return math.copysign(1, x)

def metodo_bisezione(fname, a, b, tolx):
 
    fa=fname(a)
    fb=fname(b)
    if sign(fa)*sign(fb)>0:
        print("Non è possibile applicare il metodo di bisezione \n")
        return None, None,None
    
    it = 0
    v_xk = []
    
    while abs(b-a)>tolx:
        xk = a+(b-a)/2
        v_xk.append(xk)
        it += 1
        fxk=fname(xk)
        if fxk==0:
            return xk, it, v_xk
        
        if fxk*fb<0:
            a = xk 
            fa= fxk 
        elif fxk*fa<0:
            b = xk
            fb=fxk
    return xk, it, v_xk

def falsa_posizione(fname,a,b,tolx,tolf,maxit):
    fa=fname(a)
    fb=fname(b)
    if sign(fa)*sign(fb)>0:
       print("Metodo di bisezione non applicabile")
       return None,None,None

    it=0
    v_xk=[]
    fxk=1+tolf
    errore=1+tolx
    xprec=a
    while it<=maxit and errore>=tolx and abs(fxk)>=tolf:
        xk= a-fa*(b-a)/(fb-fa)
        v_xk.append(xk)
        it+=1
        fxk=fname(xk)
        if fxk==0:
            return xk,it,v_xk

        if fxk*fa<0:
           b= xk
           fb= fxk
        elif fxk*fb<0:
           a=xk
           fa=fxk
        if xk!=0:
            errore= abs(xk-xprec)/abs(xk)
        else:
            errore=abs(xk-xprec)
        xprec=xk
    return xk,it,v_xk

def corde(fname,coeff_ang,x0,tolx,tolf,nmax):
    
    # coeff_ang è il coefficiente angolare della retta che rimane fisso per tutte le iterazioni
    xk=[]
    
    it=0
    errorex=1+tolx
    erroref=1+tolf
    while errorex>=tolx and erroref>=tolf and it<nmax:
       
       fx0=f(x0)
       d=fx0/coeff_ang
      
       x1=x-d
       fx1=f(x1)
       if x1!=0:
            errorex=abs(d)/x1 
       else:
            errorex=abs(d)
       
       erroref=abs(fx1)
       
       x0=x1
       it=it+1
       xk.append(x1)
      
    if it==nmax:
        print('Corde : raggiunto massimo numero di iterazioni \n')

    return x1,it,xk
    
def newton(fname,fpname,x0,tolx,tolf,nmax):
  
    xk=[]
   
    it=0
    errorex=1+tolx
    erroref=1+tolf
    while it<nmax and errorex>=tolx and erroref>=tolf:
       
        fx0=fname(x0)
        fpx0=fpname(x0)
        if fpx0<=np.spacing(1):
            print(" derivata prima nulla in x0")
            return None, None,None
        d=fx0/fpx0
        
        x1=x0-d
        fx1=fname(x1)
        erroref=np.abs(fx1)
        if x1!=0:
            errore=abs(d)/abs(x1)
        else:
            errore=abs(d)            
        it=it+1
        x0=x1
        xk.append(x1)
      
    if it==nmax:
        print('Newton: raggiunto massimo numero di iterazioni \n')
        
    
    return x1,it,xk

def newton_modificato(fname,fpname,m,x0,tolx,tolf,nmax):
  
    #m è la molteplicità dello zero
    
    xk=[]
   
    it=0
    errorex=1+tolx
    erroref=1+tolf
    while it<nmax and errorex>=tolf and errorex>=tolx:
       
        fx0=fname(x0)
        fpx0=fpname(x0)
        if fpx0<=np.spacing(1):
            print(" derivata prima nulla in x0")
            return None, None,None
        d=fx0/fpx0
        
        x1=x0-d
        fx1=fname(x1)
        erroref=np.abs(fx1)
        if x1!=0:
            errore=abs(d)/abs(x1)
        else:
            errore=abs(d)/abs(x1)
        
        it=it+1
        x0=x1
        xk.append(x1)
    
    if it==nmax:
        print('Newton modificato: raggiunto massimo numero di iterazioni \n')
    

    return x1,it,xk

def secanti(fname,xm1,x0,tolx,tolf,nmax):
    xk=[]
        
    it=0
    errorex=1+tolx
    erroref=1+tolf
    while errorex>=tolx and erroref>=tolf and it<nmax:        
        fxm1=f(xm1)
        fx0=f(x0) 
        d=f(x0)*(x0-xm1)/(fx0-fxm1)
        
        x1=x0-d
      
        fx1=fname(x1)
        xk.append(x1);
        if x1!=0:
            errorex=abs(d)/abs(x1)
        else:
            errorex=abs(d)
            
        erroref=abs(fx1)
        xm1=x0
        x0=x1
        
        it=it+1;
       
   
    if it==nmax:
       print('Secanti: raggiunto massimo numero di iterazioni \n')
    
    return x1,it,xk
    
def stima_ordine(xk,iterazioni):
     #Vedi dispensa allegata per la spiegazione

      k=iterazioni-4
      p=np.log(abs(xk[k+2]-xk[k+3])/abs(xk[k+1]-xk[k+2]))/np.log(abs(xk[k+1]-xk[k+2])/abs(xk[k]-xk[k+1]));
     
      ordine=p
      return ordine


#Soluzione di sistemi di equazioni non lineari
def newton_raphson(initial_guess, F_numerical, J_Numerical, tolX, tolF, max_iterations):

    X= np.array(initial_guess, dtype=float)
    
    it=0
    
    erroreF=1+tolF
    erroreX=1+tolX
    
    errore=[]
    
    while erroreX>=tolX and erroreF>=tolF and it<max_iterations:
        
        jx = J_Numerical(X[0],X[1])
        
        if np.linalg.det(jx):
            print("La matrice dello Jacobiano calcolata nell'iterato precedente non è a rango massimo")
            return None, None,None
        
        fx = F_numerical(X[0],X[1])
        fx = fx.squeeze() 
        
        s = np.linalg.solve(jx,-fx)
        
        Xnew=s+X
        
        normaXnew=np.linalg.norm(Xnew,1)
        if normaXnew !=0:
            erroreX=np.linalg.norm(s,1)/normaXnew
        else:
            erroreX=np.linalg.norm(s,1)
        
        errore.append(erroreX)
        fxnew=f(Xnew[0],Xnew[1])
        erroreF= np.linalg.norm(fxnew.squeeze(),1)
        X=Xnew
        it=it+1
    
    return X,it,errore

def newton_raphson_corde(initial_guess, F_numerical, J_Numerical, tolX, tolF, max_iterations):
    
    X= np.array(initial_guess, dtype=float)
    
   

    it=0
    
    erroreF=1+tolF
    erroreX=1+tolX
    
    errore=[]
    
    jx = J_Numerical(X[0],X[1])
        
    if np.linalg.det(jx):
        print("La matrice dello Jacobiano calcolata nell'iterato precedente non è a rango massimo")
        return None, None,None
        
    while erroreF>=tolF and erroreX>=erroreX and it<max_iterations:
        
        fx = F_numerical(X[0],X[1])
        fx = fx.squeeze() 
        
        s = np.linalg.solve(jx,-fx)
        
        Xnew=X+s
        
        normaXnew=np.linalg.norm(Xnew,1)
        if normaXnew !=0:
            erroreX=np.linalg.norm(s,1)/normaXnew 
        else:
            erroreX=np.linalg.norm(s,1)
        
        errore.append(erroreX)
        fxnew=F_numerical(Xnew[0],Xnew[1])
        erroreF= np.linalg.norm(fxnew.squeeze(),1)
        X=Xnew
        it=it+1

    return X,it,errore


def newton_raphson_sham(initial_guess, update, F_numerical, J_Numerical, tolX, tolF, max_iterations):
    
    X= np.array(initial_guess, dtype=float)

    it=0
    
    erroreF=1+tolF
    erroreX=1+tolX
    
    errore=[]
    
    while erroreF>=tolF and erroreX>=tolX and it<max_iterations:
        
        if it%update==0:
            jx = J_Numerical(X[0],X[1])
        
            if np.linalg.det(jx)==0:
                print("La matrice dello Jacobiano calcolata nell'iterato precedente non è a rango massimo")
                return None, None,None
        
        fx = F_numerical(X[0],X[1])
        fx = fx.squeeze() 
        
        s = np.linalg.solve(jx,-fx)
        
        Xnew=X+s
        
        normaXnew=np.linalg.norm(Xnew,1)
        if normaXnew !=0:
            erroreX=np.linalg.norm(s,1)/normaXnew 
        else:
            erroreX=np.linalg.norm(s,1)
        
        errore.append(erroreX)
        fxnew=F_numerical(Xnew[0],Xnew[1])
        erroreF= np.linalg.norm(fxnew.squeeze(),1)
        X=Xnew
        it=it+1
    
    return X,it,errore

#Minimo di una funzion enon lineare

def newton_raphson_minimo(initial_guess, grad_func, Hessian_func, tolX, tolF, max_iterations):
    

    X= np.array(initial_guess, dtype=float)
    
    it=0
    
    erroreF=1+tolX
    erroreX=1+tolF
    
    errore=[]
    
    while erroreF>=tolF and erroreX>=tolX and it<max_iterations:
        
        Hx = Hessian_func(X[0],X[1])
        
        if np.linalg.det(Hx)==0:
            print("La matrice Hessiana calcolata nell'iterato precedente non è a rango massimo")
            return None, None,None
        
        gfx =  grad_func(X[0],X[1]) 
        gfx = gfx.squeeze() 
        
        s = np.linalg.solve(Hx,-gfx)
        
        Xnew=X+s 
        
        normaXnew=np.linalg.norm(Xnew,1)
        if normaXnew!=0:
            erroreX=np.linalg.norm(s,1)/normaXnew
        else:
            erroreX=np.linalg.norm(s,1)
            
        errore.append(erroreX)
        gfxnew=grad_func(Xnew[0],Xnew[1])
        erroreF= np.linalg.norm(gfxnew.squeeze(),1)
        X=Xnew
        it=it+1
    
    return X,it,errore

#Metodi Iterativi basati sullo splitting della matrice: jacobi, gauss-Seidel - Gauss_seidel SOR
def jacobi(A,b,x0,toll,it_max):
    errore=1000
    d=np.diag(A)
    n=A.shape[0]
    invM=np.diag(1/d)
    E= np.tril(A,-1)
    F= np.triu(A,1)
    N= -(E+F)
    T=invM@N 
    autovalori=np.linalg.eigvals(T)
    raggiospettrale=np.max(autovalori)
    print("raggio spettrale jacobi", raggiospettrale)
    it=0
    
    er_vet=[]
    while it<=it_max and errore>=toll:
        x=(N@x0-b)/d.reshape(n,1)
        errore=np.linalg.norm(x-x0)/np.linalg.norm(x)
        er_vet.append(errore)
        x0=x.copy()
        it=it+1
    return x,it,er_vet


def gauss_seidel(A,b,x0,toll,it_max):
    errore=1000
    d=np.diag(A)
    D=np.diag(A)
    E=np.tril(A,-1)
    F=np.triu(A,1)
    M=E+D 
    N=-F
    T=np.linalg.inv(M)@N 
    autovalori=np.linalg.eigvals(T)
    raggiospettrale=np.max(autovalori)
    
    print("raggio spettrale Gauss-Seidel ",raggiospettrale)
    it=0
    er_vet=[]
    while errore>=toll and it<it_max:
        x=st.Lsolve(M,b-F@x0)
        errore=np.linalg.norm(x-x0)/np.linalg.norm(x)
        er_vet.append(errore)
        x0=x.copy()
        it=it+1
    return x,it,er_vet

def gauss_seidel_sor(A,b,x0,toll,it_max,omega):
    errore=1000
    d=np.diag(A)
    D=np.diag(d)
    E=np.tril(A,-1)
    F=np.triu(A,1)
    Momega=D+omega*E
    Nomega=(1-omega)*D-omega*F
    T=np.linalg.inv(Momega)@Nomega
    autovalori=np.linalg.eigvals(T)
    raggiospettrale=np.max(autovalori)
    print("raggio spettrale Gauss-Seidel SOR ", raggiospettrale)
    
    M=D+E
    N=-F 
    it=0
    xold=x0.copy()
    xnew=x0.copy()
    er_vet=[]
    while it<=it_max and errore>=toll:
        
        xtilde=st.Lsolve(M,b-F@xold)
        xnew=(1-omega)*xold+omega*xtilde
        errore=np.linalg.norm(xnew-xold)/np.linalg.norm(xnew)
        er_vet.append(errore)
        xold=xnew.copy()
        it=it+1
    return xnew,it,er_vet


#Metodi di Discesa

def steepestdescent(A,b,x0,itmax,tol):
    n,m=A.shape
    if n!=m:
        print("Matrice non quadrata")
        return [],[]
    
    
   # inizializzare le variabili necessarie
    x = x0
     
    r = A@x0-b
    p = -r 
    it = 0
    nb=np.linalg.norm(b)
    errore=np.linalg.norm(r)/nb
    vec_sol=[]
    vec_sol.append(x.copy())
    vet_r=[]
    vet_r.append(errore)
     
# utilizzare il metodo del gradiente per trovare la soluzione
    while errore>=tol and it<itmax: 
        it=it+1
        Ap=A@p 
       
        alpha = (r.T@r)/(p.T@Ap)
                
        x = x+alpha*p
        
         
        vec_sol.append(x.copy())
        r=r+alpha*Ap
        errore=np.linalg.norm(r)/nb
        vet_r.append(errore)
        p =-r
        
    iterates_array = np.vstack([arr.T for arr in vec_sol])
    return x,vet_r,iterates_array,it

def conjugate_gradient(A,b,x0,itmax,tol):
    n,m=A.shape
    if n!=m:
        print("Matrice non quadrata")
        return [],[]
    
    
   # inizializzare le variabili necessarie
    x = x0.copy()
    
    r = A@x-b
    p = -r 
    it = 0
    nb=np.linalg.norm(b)
    errore=np.linalg.norm(r)/nb
    vec_sol=[]
    vec_sol.append(x0.copy())
    vet_r=[]
    vet_r.append(errore)
# utilizzare il metodo del gradiente coniugato per calcolare la soluzione
    while errore>=tol:
        it=it+1
        Ap=A@p
        save=r.T@r
        alpha = save/(p.T@Ap) 
        x = x+alpha*p
        vec_sol.append(x.copy())
        r= r+alpha*Ap
        gamma=(r.T@r)/save
        errore=np.linalg.norm(r)/nb
        vet_r.append(errore)
        p = -r + gamma*p
   
    iterates_array = np.vstack([arr.T for arr in vec_sol])
    return x,vet_r,iterates_array,it

#Soluzione di sistemi sovradeterminati

def eqnorm(A,b):
 
    G= A.T@A  
    f=A.T@b 
    
    L=np.linalg.cholesky(G,lower=True)
    U=L.T

    y,flag=st.Lsolve(L,f)
    if flag==0:
        x,flag=st.Usolve(U,y)
    
    return x


def qrLS(A,b):
    n=A.shape[1]  # numero di colonne di A
    Q,R=spLin.qr(A)
    h=Q.T@b
    x,flag=st.Usolve(Q[:n,:n],h[:n])
    residuo=np.linalg.norm(h[n:])
    return x,residuo


def SVDLS(A,b):
    m,n=A.shape  #numero di righe e  numero di colonne di A
    U,s,VT=spLin.svd(A)  
    
    V=VT.T
    thresh=np.spacing(1)*m*s[0] ##Calcolo del rango della matrice, numero dei valori singolari maggiori di una soglia
    k= np.count_nonzero(s>thresh)
    
    d=U.T@b
    d1=d[:k].reshape(k,1) 
    s1=d[:k].reshape(k,1)
    
    c=d1/s1
    x=V[:,:k]@c
    residuo=np.linalg.norm(d[k:])**2
    return x,residuo
     

#-----------Interpolazione

def plagr(xnodi,j):
    
    xzeri=np.zeros_like(xnodi)
    n=xnodi.size
    xzeri=np.append(xnodi[:j],xnodi[j+1:])
    
    num=np.poly(xzeri)
    den=np.polyval(num,xnodi[j])
    
    p=num/den
    
    return p



def InterpL(x, y, xx):
     
     n=x.shape
     m=xx.shape
     L=np.zeros((m,n))
     for j in range(n):
        p=plagr(x,j)
        L[:,j]=np.polyval(p,xx) 
    
     return L@y